# IMPORTAR LIBRERÍAS Y CONFIGURAR PATH Y CLIENTE

In [1]:
# Import de las librerías, api keys y funciones
import sys
import os
import pandas as pd
import numpy as np
from pathlib import Path
import sqlite3
import time
from datetime import datetime, timedelta
import re
import json
from apify_client import ApifyClient
from config import APIFY_API_KEY, DB_PATH
from src.utils.database import get_db_connection, insert_dataframe_to_table

# Configuración de paths
project_root = Path.cwd().parent.parent
sys.path.append(str(project_root))

# Configuración de Apify
client = ApifyClient(APIFY_API_KEY)

### Establecer los parámetros del scraping

In [ ]:
# Configuración ajustable
LIMITE_RESTAURANTES_PARAM = 30  # número de restaurantes a scrapear las reviews
DELAY_ENTRE_RESTAURANTES_PARAM = 5  # segundos de espera entre cada scraping

print(f"⚙️  Configuración: {LIMITE_RESTAURANTES_PARAM} restaurantes, delay {DELAY_ENTRE_RESTAURANTES_PARAM}s")

⚙️  Configuración: 30 restaurantes, delay 5s


# DEFINIR LAS FUNCIONES PARA SCRAPEAR LAS REVIEWS

In [3]:
# Obtener restaurantes para escanear
def get_restaurantes_para_escaneo(limite=2):
    """
    Obtiene restaurantes que necesitan ser escaneados (priorizando los más antiguos)
    """
    conn = get_db_connection()
    
    query = """
    SELECT id, tripadvisor_id, nombre, tripadvisor_web, fecha_escaneo_reviews
    FROM Restaurantes 
    WHERE tripadvisor_web IS NOT NULL
    ORDER BY fecha_escaneo_reviews ASC NULLS FIRST, id ASC
    LIMIT ?
    """
    
    try:
        df_restaurantes = pd.read_sql_query(query, conn, params=(limite,))
        print(f"📊 Restaurantes para escanear: {len(df_restaurantes)}")
        return df_restaurantes
    except Exception as e:
        print(f"❌ Error obteniendo restaurantes: {e}")
        return pd.DataFrame()
    finally:
        conn.close()

In [ ]:
# Función para obtener las reviews
def get_reviews_por_restaurante(tripadvisor_web, restaurante_id, tripadvisor_id, nombre_restaurante):
    """
    Función para obtener las reviews de los restaurantes pasados por parámetro.
    """
    print(f"📖 Escaneando: {nombre_restaurante}")
    
    reviews_data = []
    rating_histogram = {1: 0, 2: 0, 3: 0, 4: 0, 5: 0}
    
    # Configuración para obtener TODAS las reviews
    run_input = {
        "startUrls": [{"url": tripadvisor_web}],
        "reviewsLimit": 500,
        "language": "all",
        "proxyConfiguration": {"useApifyProxy": True},
        "includeRestaurants": True,
        #"lastReviewDate": '1 year',
        #"reviewRatings": [1,2] o ["Terrible", "Poor"]
    }
    
    try:
        run = client.actor("maxcopell/tripadvisor-reviews").call(run_input=run_input)
        dataset_items = list(client.dataset(run["defaultDatasetId"]).iterate_items())
        
        print(f"📦 Obtenidas {len(dataset_items)} reviews (todas)")
        
        # Filtrar localmente solo ratings 1-2
        for item in dataset_items:
            rating = item.get('rating', 0)
            
            # Actualizar histograma completo
            if 1 <= rating <= 5:
                rating_histogram[rating] += 1
            
            # Filtrar solo ratings 1-2
            if rating in [1, 2]:
                review_data = {
                    'restaurante_id': restaurante_id,
                    'tripadvisor_id': tripadvisor_id,
                    'review_id': item.get('id'),
                    'titulo': item.get('title'),
                    'contenido': item.get('text'),
                    'rating': rating,
                    'idioma': item.get('lang'),
                    'fecha_publicacion': item.get('publishedDate'),
                    'fecha_experiencia': item.get('travelDate'),
                    'tipo_viaje': item.get('tripType'),
                    'usuario': item.get('user', {}).get('name'),
                    'usuario_id': item.get('user', {}).get('userId'),
                    'contribuciones_usuario': item.get('user', {}).get('contributions', {}).get('totalContributions', 0),
                    'votos_utiles': item.get('helpfulVotes', 0),
                    'respuesta_empresa': item.get('ownerResponse', {}).get('text') if item.get('ownerResponse') else None,
                    'subratings': json.dumps(item.get('subratings', [])) if item.get('subratings') else None,
                    'fotos': json.dumps(item.get('photos', [])) if item.get('photos') else None
                }
                reviews_data.append(review_data)
        
        print(f"✅ {len(reviews_data)} reviews negativas (1-2 estrellas) filtradas")
        print(f"📊 Histograma completo: {rating_histogram}")
        
        return reviews_data, rating_histogram
        
    except Exception as e:
        print(f"❌ Error escaneando {nombre_restaurante}: {e}")
        return [], {1: 0, 2: 0, 3: 0, 4: 0, 5: 0}

In [5]:
# Funciones para guardar datos
def guardar_reviews_en_bd(df_reviews):
    """
    Guarda las reviews en la base de datos
    """
    if not df_reviews.empty:
        try:
            conn = get_db_connection()
            df_reviews.to_sql('Reviews', conn, if_exists='append', index=False)
            conn.close()
            print(f"✅ {len(df_reviews)} reviews guardadas en la BD")
        except Exception as e:
            print(f"❌ Error guardando reviews: {e}")
    else:
        print("📭 No hay reviews negativas para guardar")

def actualizar_restaurante_escaneo(restaurante_id, rating_histogram):
    """
    Actualiza la fecha de escaneo y el histograma de ratings para un restaurante
    """
    try:
        conn = get_db_connection()
        cursor = conn.cursor()
        
        update_query = """
        UPDATE Restaurantes 
        SET fecha_escaneo_reviews = CURRENT_TIMESTAMP,
            conteo_rating_1 = ?, conteo_rating_2 = ?, conteo_rating_3 = ?,
            conteo_rating_4 = ?, conteo_rating_5 = ?
        WHERE id = ?
        """
        
        cursor.execute(update_query, (
            rating_histogram[1], rating_histogram[2], rating_histogram[3],
            rating_histogram[4], rating_histogram[5], restaurante_id
        ))
        
        conn.commit()
        conn.close()
        print(f"✅ Restaurante {restaurante_id} actualizado con histograma")
        
    except Exception as e:
        print(f"❌ Error actualizando restaurante: {e}")

In [6]:
# Función principal de escaneo de reviews
def ejecutar_escaneo_reviews(limite_restaurantes, delay):
    """
    Función principal para ejecutar el escaneo
    """
    print(f"🎯 INICIANDO ESCANEO DE REVIEWS")
    print("=" * 60)
    
    df_restaurantes = get_restaurantes_para_escaneo(limite_restaurantes)
    
    if df_restaurantes.empty:
        print("🎉 No hay restaurantes para escanear!")
        return
    
    total_reviews_negativas = 0
    
    for i, (_, restaurante) in enumerate(df_restaurantes.iterrows(), 1):
        print(f"\n[{i}/{len(df_restaurantes)}] ", end="")
        
        reviews, rating_histogram = get_reviews_por_restaurante(
                restaurante['tripadvisor_web'],
                restaurante['id'],
                restaurante['tripadvisor_id'],
                restaurante['nombre']
            )
        
        if reviews:
            df_reviews = pd.DataFrame(reviews)
            guardar_reviews_en_bd(df_reviews)
            total_reviews_negativas += len(reviews)
        
        actualizar_restaurante_escaneo(restaurante['id'], rating_histogram)
        
        if i < len(df_restaurantes):
            print(f"⏰ Esperando {delay} segundos...")
            time.sleep(delay)
    
    print(f"\n🎯 ESCANEO COMPLETADO")
    print(f"   • Restaurantes escaneados: {len(df_restaurantes)}")
    print(f"   • Reviews negativas encontradas: {total_reviews_negativas}")

# EJECUTAR EL SCRAPEO Y ESCANEO DE LAS REVIEWS

In [7]:
# Ejecutar el escaneo de las reviews
print("\n\n🔧 EJECUTANDO EL ESCANEO DE LAS REVIEWS")
ejecutar_escaneo_reviews(limite_restaurantes=LIMITE_RESTAURANTES_PARAM, delay=DELAY_ENTRE_RESTAURANTES_PARAM)



🔧 EJECUTANDO EL ESCANEO DE LAS REVIEWS
🎯 INICIANDO ESCANEO DE REVIEWS
📍 Conectando a: /Users/administrator/MASTER_Data_Science/TFM/tfme-horeca-intelligence/tfm_database.db
📍 ¿Existe el archivo? True
✅ Conexión a la BD SQLite exitosa.
📊 Restaurantes para escanear: 30

[1/30] 📖 Escaneando: Mamma Mia Restaurant And Bar


[apify.tripadvisor-reviews runId:2mJPKavxFm78FXmrP] -> Status: RUNNING, Message: 
[apify.tripadvisor-reviews runId:2mJPKavxFm78FXmrP] -> 2025-11-01T10:29:35.603Z ACTOR: Pulling container image of build LcGpCH3n6xCuwrm5y from registry.
[apify.tripadvisor-reviews runId:2mJPKavxFm78FXmrP] -> 2025-11-01T10:29:35.605Z ACTOR: Creating container.
[apify.tripadvisor-reviews runId:2mJPKavxFm78FXmrP] -> 2025-11-01T10:29:35.651Z ACTOR: Starting container.
[apify.tripadvisor-reviews runId:2mJPKavxFm78FXmrP] -> 2025-11-01T10:29:35.822Z Will run command: xvfb-run -a -s "-ac -screen 0 1920x1080x24+32 -nolisten tcp" /bin/sh -c node dist/entrypoint.js
[apify.tripadvisor-reviews runId:2mJPKavxFm78FXmrP] -> 2025-11-01T10:29:37.851Z INFO  System info {"apifyVersion":"3.1.15","apifyClientVersion":"2.8.4","crawleeVersion":"3.7.2","osType":"Linux","nodeVersion":"v20.19.5"}
[apify.tripadvisor-reviews runId:2mJPKavxFm78FXmrP] -> 2025-11-01T10:29:38.414Z INFO  Input validation OK
[apify.tripadvisor-reviews runI

📦 Obtenidas 1 reviews (todas)
✅ 0 reviews negativas (1-2 estrellas) filtradas
📊 Histograma completo: {1: 0, 2: 0, 3: 0, 4: 0, 5: 1}
📍 Conectando a: /Users/administrator/MASTER_Data_Science/TFM/tfme-horeca-intelligence/tfm_database.db
📍 ¿Existe el archivo? True
✅ Conexión a la BD SQLite exitosa.
✅ Restaurante 81 actualizado con histograma
⏰ Esperando 5 segundos...

[2/30] 📖 Escaneando: Cuina Artesana


[apify.tripadvisor-reviews runId:6Hc7OxuJhhyTyAmlS] -> Status: RUNNING, Message: 
[apify.tripadvisor-reviews runId:6Hc7OxuJhhyTyAmlS] -> 2025-11-01T10:29:53.681Z ACTOR: Pulling container image of build LcGpCH3n6xCuwrm5y from registry.
[apify.tripadvisor-reviews runId:6Hc7OxuJhhyTyAmlS] -> 2025-11-01T10:29:53.683Z ACTOR: Creating container.
[apify.tripadvisor-reviews runId:6Hc7OxuJhhyTyAmlS] -> 2025-11-01T10:29:54.290Z ACTOR: Starting container.
[apify.tripadvisor-reviews runId:6Hc7OxuJhhyTyAmlS] -> 2025-11-01T10:29:54.513Z Will run command: xvfb-run -a -s "-ac -screen 0 1920x1080x24+32 -nolisten tcp" /bin/sh -c node dist/entrypoint.js
[apify.tripadvisor-reviews runId:6Hc7OxuJhhyTyAmlS] -> 2025-11-01T10:29:57.061Z INFO  System info {"apifyVersion":"3.1.15","apifyClientVersion":"2.8.4","crawleeVersion":"3.7.2","osType":"Linux","nodeVersion":"v20.19.5"}
[apify.tripadvisor-reviews runId:6Hc7OxuJhhyTyAmlS] -> 2025-11-01T10:29:57.213Z INFO  Input validation OK
[apify.tripadvisor-reviews runI

📦 Obtenidas 2 reviews (todas)
✅ 0 reviews negativas (1-2 estrellas) filtradas
📊 Histograma completo: {1: 0, 2: 0, 3: 0, 4: 0, 5: 2}
📍 Conectando a: /Users/administrator/MASTER_Data_Science/TFM/tfme-horeca-intelligence/tfm_database.db
📍 ¿Existe el archivo? True
✅ Conexión a la BD SQLite exitosa.
✅ Restaurante 82 actualizado con histograma
⏰ Esperando 5 segundos...

[3/30] 📖 Escaneando: Cua de Gall


...

📦 Obtenidas 116 reviews (todas)
✅ 82 reviews negativas (1-2 estrellas) filtradas
📊 Histograma completo: {1: 71, 2: 11, 3: 11, 4: 15, 5: 8}
📍 Conectando a: /Users/administrator/MASTER_Data_Science/TFM/tfme-horeca-intelligence/tfm_database.db
📍 ¿Existe el archivo? True
✅ Conexión a la BD SQLite exitosa.
✅ 82 reviews guardadas en la BD
📍 Conectando a: /Users/administrator/MASTER_Data_Science/TFM/tfme-horeca-intelligence/tfm_database.db
📍 ¿Existe el archivo? True
✅ Conexión a la BD SQLite exitosa.
✅ Restaurante 108 actualizado con histograma
⏰ Esperando 5 segundos...

[29/30] 📖 Escaneando: El Museu del Vi


[apify.tripadvisor-reviews runId:mGtleGj9vwPwLeKYU] -> Status: RUNNING, Message: 
[apify.tripadvisor-reviews runId:mGtleGj9vwPwLeKYU] -> 2025-11-01T10:37:52.776Z ACTOR: Pulling container image of build LcGpCH3n6xCuwrm5y from registry.
[apify.tripadvisor-reviews runId:mGtleGj9vwPwLeKYU] -> 2025-11-01T10:37:52.778Z ACTOR: Creating container.
[apify.tripadvisor-reviews runId:mGtleGj9vwPwLeKYU] -> 2025-11-01T10:37:52.860Z ACTOR: Starting container.
[apify.tripadvisor-reviews runId:mGtleGj9vwPwLeKYU] -> 2025-11-01T10:37:53.056Z Will run command: xvfb-run -a -s "-ac -screen 0 1920x1080x24+32 -nolisten tcp" /bin/sh -c node dist/entrypoint.js
[apify.tripadvisor-reviews runId:mGtleGj9vwPwLeKYU] -> 2025-11-01T10:37:54.055Z INFO  System info {"apifyVersion":"3.1.15","apifyClientVersion":"2.8.4","crawleeVersion":"3.7.2","osType":"Linux","nodeVersion":"v20.19.5"}
[apify.tripadvisor-reviews runId:mGtleGj9vwPwLeKYU] -> 2025-11-01T10:37:54.235Z INFO  Input validation OK
[apify.tripadvisor-reviews runI

📦 Obtenidas 216 reviews (todas)
✅ 70 reviews negativas (1-2 estrellas) filtradas
📊 Histograma completo: {1: 33, 2: 37, 3: 45, 4: 63, 5: 38}
📍 Conectando a: /Users/administrator/MASTER_Data_Science/TFM/tfme-horeca-intelligence/tfm_database.db
📍 ¿Existe el archivo? True
✅ Conexión a la BD SQLite exitosa.
✅ 70 reviews guardadas en la BD
📍 Conectando a: /Users/administrator/MASTER_Data_Science/TFM/tfme-horeca-intelligence/tfm_database.db
📍 ¿Existe el archivo? True
✅ Conexión a la BD SQLite exitosa.
✅ Restaurante 109 actualizado con histograma
⏰ Esperando 5 segundos...

[30/30] 📖 Escaneando: La Piadina


[apify.tripadvisor-reviews runId:RDxbq3HyrrVD43NdE] -> Status: RUNNING, Message: 
[apify.tripadvisor-reviews runId:RDxbq3HyrrVD43NdE] -> 2025-11-01T10:38:17.317Z ACTOR: Pulling container image of build LcGpCH3n6xCuwrm5y from registry.
[apify.tripadvisor-reviews runId:RDxbq3HyrrVD43NdE] -> 2025-11-01T10:38:17.319Z ACTOR: Creating container.
[apify.tripadvisor-reviews runId:RDxbq3HyrrVD43NdE] -> 2025-11-01T10:38:17.357Z ACTOR: Starting container.
[apify.tripadvisor-reviews runId:RDxbq3HyrrVD43NdE] -> 2025-11-01T10:38:17.522Z Will run command: xvfb-run -a -s "-ac -screen 0 1920x1080x24+32 -nolisten tcp" /bin/sh -c node dist/entrypoint.js
[apify.tripadvisor-reviews runId:RDxbq3HyrrVD43NdE] -> 2025-11-01T10:38:19.542Z INFO  System info {"apifyVersion":"3.1.15","apifyClientVersion":"2.8.4","crawleeVersion":"3.7.2","osType":"Linux","nodeVersion":"v20.19.5"}
[apify.tripadvisor-reviews runId:RDxbq3HyrrVD43NdE] -> 2025-11-01T10:38:19.686Z INFO  Input validation OK
[apify.tripadvisor-reviews runI

📦 Obtenidas 5 reviews (todas)
✅ 0 reviews negativas (1-2 estrellas) filtradas
📊 Histograma completo: {1: 0, 2: 0, 3: 3, 4: 0, 5: 2}
📍 Conectando a: /Users/administrator/MASTER_Data_Science/TFM/tfme-horeca-intelligence/tfm_database.db
📍 ¿Existe el archivo? True
✅ Conexión a la BD SQLite exitosa.
✅ Restaurante 110 actualizado con histograma

🎯 ESCANEO COMPLETADO
   • Restaurantes escaneados: 30
   • Reviews negativas encontradas: 277


# VERIFICAR LA CARGA DEL HISTOGRAMA EN LOS RESTAURANTES ESCANEADOS

In [8]:
# Verificar histograma en la BD
def verificar_histograma():
    """
    Verifica que el histograma se haya guardado correctamente
    """
    conn = get_db_connection()
    
    query = """
    SELECT id, nombre, conteo_rating_1, conteo_rating_2, conteo_rating_3, 
           conteo_rating_4, conteo_rating_5, fecha_escaneo_reviews
    FROM Restaurantes 
    WHERE fecha_escaneo_reviews IS NOT NULL
    ORDER BY fecha_escaneo_reviews DESC
    LIMIT 5
    """
    
    try:
        df = pd.read_sql_query(query, conn)
        print("📊 HISTOGRAMA EN BASE DE DATOS:")
        print("=" * 50)
        if not df.empty:
            for _, row in df.iterrows():
                print(f"🍽️  {row['nombre']}")
                print(f"   ⭐1: {row['conteo_rating_1']} | ⭐2: {row['conteo_rating_2']} | ⭐3: {row['conteo_rating_3']}")
                print(f"   ⭐4: {row['conteo_rating_4']} | ⭐5: {row['conteo_rating_5']}")
                print(f"   📅 Último escaneo: {row['fecha_escaneo_reviews']}")
                print("-" * 30)
        else:
            print("❌ No hay datos de histograma")
            
    except Exception as e:
        print(f"❌ Error verificando histograma: {e}")
    finally:
        conn.close()

# Ejecutar verificación
verificar_histograma()

📍 Conectando a: /Users/administrator/MASTER_Data_Science/TFM/tfme-horeca-intelligence/tfm_database.db
📍 ¿Existe el archivo? True
✅ Conexión a la BD SQLite exitosa.
📊 HISTOGRAMA EN BASE DE DATOS:
🍽️  La Piadina
   ⭐1: 0 | ⭐2: 0 | ⭐3: 3
   ⭐4: 0 | ⭐5: 2
   📅 Último escaneo: 2025-11-01 10:38:28
------------------------------
🍽️  El Museu del Vi
   ⭐1: 33 | ⭐2: 37 | ⭐3: 45
   ⭐4: 63 | ⭐5: 38
   📅 Último escaneo: 2025-11-01 10:38:12
------------------------------
🍽️  Catalano Taverna
   ⭐1: 71 | ⭐2: 11 | ⭐3: 11
   ⭐4: 15 | ⭐5: 8
   📅 Último escaneo: 2025-11-01 10:37:47
------------------------------
🍽️  Sushi Bar Time Coffee House
   ⭐1: 0 | ⭐2: 0 | ⭐3: 0
   ⭐4: 0 | ⭐5: 2
   📅 Último escaneo: 2025-11-01 10:37:26
------------------------------
🍽️  Ven Tip
   ⭐1: 0 | ⭐2: 0 | ⭐3: 0
   ⭐4: 0 | ⭐5: 5
   📅 Último escaneo: 2025-11-01 10:37:09
------------------------------
